## Imports

In [ ]:
import xarray as xr
import netCDF4 as nc
import pandas as pd

import matplotlib.pylab as plt
import numpy as np
import matplotlib.cm as cm
import cmocean.cm as cmo


import numcodecs
import scipy.stats as stats

from getpass import getuser # Libaray to copy things
from pathlib import Path # Object oriented libary to deal with paths
from dask.utils import format_bytes
from distributed import Client, progress, wait # Libaray to orchestrate distributed resources
#from dask_jobqueue import SLURMCluster # Setting up distributed memories via slurm
import numpy as np # Pythons standard array library

import dask.distributed
import multiprocessing
from subprocess import run, PIPE
import sys
import os
import matplotlib.ticker as ticker

In [ ]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
from matplotlib import pyplot as plt # Standard Plotting library
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
from cartopy import crs as ccrs # Cartography library
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import matplotlib.tri as tri

In [ ]:
#'''Provides information about the CPU and starts the dask client'''
ncpu = multiprocessing.cpu_count()
processes = False
nworker = 1
threads = ncpu // nworker
print(f"Number of CPUs: {ncpu}, number of threads: {threads}, number of workers: {nworker}, processes: {processes}")
client = Client(processes=processes,
 threads_per_worker=threads,
 n_workers=nworker,
 memory_limit='500GB'
 )

In [ ]:
client

In [ ]:
month = "annual"

if month == 'DJF':
    months = [12,1,2]
elif month == 'MAM':
    months = [3,4,5]
elif month == 'JJA':
    months = [6,7,8]
elif month == 'SON':
    months = [9,10,11]
elif month == 'annual':
    months = []    

latitude = [35,45]
longitude = [-10,35.5]

## CESM2

cmip_fr

In [ ]:
data_path = Path('/home/jwille/xenon_data/NextGEMS/CESM2/precip/24H')
files = sorted([str(f) for f in data_path.glob(f'24hr_b.e21.BHISTsmbb*.nc')])[:]
dataset = xr.open_mfdataset(files, engine='netcdf4',combine='by_coords')['PRECT']
mask_obs_1 = dataset.sel(lon=np.arange(0,181.25,1.25))
mask_obs_2 = dataset.sel(lon=np.arange(181.25,360,1.25))
mask_obs_2['lon'] = mask_obs_2.lon - 360
dataset = xr.concat([mask_obs_2,mask_obs_1],dim='lon')
tp = dataset * 3600 * 1000

In [ ]:
if month != 'annual':
    tp = tp.where(tp.time.dt.month.isin(months), drop=True)

In [ ]:
cesm2 = tp.where(((tp.lat>=34) & (tp.lat<=46)) & ((tp.lon>= -11) & (tp.lon<=36)),drop=True)

map

## ERA5

pr_t

In [ ]:

dataset = xr.open_dataset('/home/jwille/xenon_data/NextGEMS/era5/precip/cesm2_interp_precip_1950_2022_era5.nc')


In [ ]:
#Get the PRECT variable
precip = dataset.tp

#Transform rate into total by multiplying by 86400s
#df['vel'] = np.sqrt(df['u'].values**2 + df['v'].values*df['v'].values)
precip = precip * 1000

In [ ]:
if month != 'annual':
    precip = precip.where(precip.time.dt.month.isin(months), drop=True)

In [ ]:
indexer = ((precip.lat>=latitude[0]) & (precip.lat<=latitude[1]) & (precip.lon>=longitude[0]) & (precip.lon<=longitude[1])).compute()
era5 = precip.where(indexer,drop=True)

Map

## EOBS

frame

In [ ]:
dataset = xr.open_dataset('/home/jwille/xenon_data/NextGEMS/EOBS/cesm2_interp_eobs.nc')


In [ ]:
#Get the PRECT variable
precip = dataset.rr




In [ ]:
if month != 'annual':
    precip = precip.where(precip.time.dt.month.isin(months), drop=True)

In [ ]:
indexer = ((precip.lat>=latitude[0]) & (precip.lat<=latitude[1]) & (precip.lon>=longitude[0]) & (precip.lon<=longitude[1])).compute()
eobs = precip.where(indexer,drop=True)

Map

## Icon 

In [ ]:
data = xr.open_dataset('/home/jwille/xenon_data/NextGEMS/ICON/precip/ngc3028/zoom7/1H_interp/cesm2_interp_precip_1H_2020_2025_ICON_newlons.nc')
dataset = data.pr
dataset = dataset * 60 * 30
dataset = dataset.sel(time=slice("2020-01-01", "2024-12-31"))
precip = dataset.resample(time='1D').sum()

In [ ]:
if month != 'annual':
    precip = precip.where(precip.time.dt.month.isin(months), drop=True)


In [ ]:
indexer = ((precip.lat>=latitude[0]) & (precip.lat<=latitude[1]) & (precip.lon>=longitude[0]) & (precip.lon<=longitude[1])).compute()
icon = precip.where(indexer,drop=True)

## IFS

In [ ]:
data = xr.open_dataset('/home/jwille/xenon_data/NextGEMS/IFS/precip/IFS_4.4-FESOM_5/025deg/1H_interp/cesm2_interp_precip_1H_2020_2024_IFS_newlons.nc')
dataset = data.tp
dataset = dataset * 1000 
precip = dataset.resample(time='1D').sum()

In [ ]:
if month != 'annual':
    precip = precip.where(precip.time.dt.month.isin(months), drop=True)

In [ ]:
indexer = ((precip.lat>=latitude[0]) & (precip.lat<=latitude[1]) & (precip.lon>=longitude[0]) & (precip.lon<=longitude[1])).compute()
ifs = precip.where(indexer,drop=True)

In [ ]:
#print(np.nanmax(ifs))
#print(np.nanmax(icon))
print(np.nanmax(eobs))

## IMERG

In [ ]:
data_path = Path('/home/jwille/xenon_data/NextGEMS/IMERG/clean/CESM2_res/concat')

files = sorted([str(f) for f in data_path.glob(f'new_lons*.nc')])[:]
dataset = xr.open_mfdataset(files, engine='netcdf4',combine='by_coords')['precipitation']    
dataset = dataset / 2    
dataset = dataset.sel(time=slice("2001-01-01", "2022-12-31"))
dataset = dataset.resample(time='1D').sum(skipna=False)

In [ ]:
if month != 'annual':
    precip = precip.where(precip.time.dt.month.isin(months), drop=True)

In [ ]:
indexer = ((precip.lat>=latitude[0]) & (precip.lat<=latitude[1]) & (precip.lon>=longitude[0]) & (precip.lon<=longitude[1])).compute()
imerg = precip.where(indexer,drop=True)

Map

## Plots

In [ ]:
#cycle3


bins1 = np.arange(cesm2.min(), cesm2.max() + 2, 2)
bins2 = np.arange(era5.min(), era5.max() + 2, 2)
bins3 = np.arange(eobs.min(), eobs.max() + 2, 2)
bins4 = np.arange(icon.min(), icon.max() + 2, 2)
bins5 = np.arange(ifs.min(), ifs.max() + 2, 2)
bins6 = np.arange(imerg.min(), imerg.max() + 2, 2)



In [ ]:
# probability density function (PDF) 

#icon_line = stats.gaussian_kde(icon.values.flatten()[np.logical_not(np.isnan(icon.values.flatten()))])
#era5_line = stats.gaussian_kde(era5.values.flatten()[np.logical_not(np.isnan(era5.values.flatten()))])
#ifs_line = stats.gaussian_kde(ifs.values.flatten()[np.logical_not(np.isnan(ifs.values.flatten()))])
#eobs_line = stats.gaussian_kde(eobs.values.flatten()[np.logical_not(np.isnan(eobs.values.flatten()))])
#cesm2_line = stats.gaussian_kde(cesm2.values.flatten()[np.logical_not(np.isnan(cesm2.values.flatten()))])



In [ ]:




fig = plt.figure(figsize = (16,8)) 
plt.rc('axes', titlesize=18)
plt.rc('axes', labelsize=14)

colors = ['#FF8C69', '#7FFF7F', '#FFC300', '#FF69B4', '#6BB7EC', '#C5A5CF80','#AEC6CF']

#plt.hist(pr_t.values.flatten(), bins = bins2, density = True, log = True, color = pink, alpha=1, label='CERRA')
#plt.hist(cmip_fr.values.flatten(), bins = bins1, density = True, log = True, color = 'pink', alpha=0.6, label='CMIP6')

#plt.hist(icon.values.flatten(), bins = bins4, density = True, log = True, color = colors[0], alpha=0.5, label='ICON',histtype='bar', rwidth=0.8)
#plt.hist(era5.values.flatten(), bins = bins2, density = True, log = True, color = colors[1], alpha=0.5, label='ERA5',histtype='bar', rwidth=0.8)
#plt.hist(ifs.values.flatten(), bins = bins5, density = True, log = True, color = colors[2], alpha=0.5, label='IFS_4.4-FESOM_5',histtype='bar', rwidth=0.8)
#plt.hist(eobs.values.flatten(), bins = bins3, density = True, log = True, color = colors[5], alpha=0.7, label='E-OBS',histtype='bar', rwidth=0.8)
#plt.hist(cesm2.values.flatten(), bins = bins1, density = True, log = True, color = colors[4], alpha=1.0,fc='None', ec=colors[4],label='CESM2')

plt.hist(icon.values.flatten(), bins = bins4, density = True, log = True, color = colors[0], alpha=1.0, label='ICON',histtype='step', rwidth=0.8)
plt.hist(era5.values.flatten(), bins = bins2, density = True, log = True, color = colors[1], alpha=1.0, label='ERA5',histtype='step', rwidth=0.8)
plt.hist(ifs.values.flatten(), bins = bins5, density = True, log = True, color = colors[2], alpha=1.0, label='IFS',histtype='step', rwidth=0.8)
plt.hist(eobs.values.flatten(), bins = bins3, density = True, log = True, color = colors[5], alpha=1.0, label='E-OBS',histtype='step', rwidth=0.8)
plt.hist(imerg.values.flatten(), bins = bins6, density = True, log = True, color = colors[6], alpha=1.0, label='IMERG',histtype='step', rwidth=0.8)

n, x, _ = plt.hist(cesm2.values.flatten(), bins = bins1, density = True, log = True, color = colors[4], alpha=1.0,histtype='step', rwidth=0.8,label='CESM2')

#plt.plot(x, icon_line(x),color = colors[0])
#plt.plot(x, era5_line(x),color = colors[1])
#plt.plot(x, ifs_line(x),color = colors[2])
#plt.plot(x, eobs_line(x),color = colors[5])
#plt.plot(x, cesm2_line(x),color = colors[4])

plt.xlabel('mm day' + r'$^{-1}$',fontsize=20)
plt.ylabel('Probability',fontsize=20)
plt.title('Mediterranean daily precipitation (CESM2 grid), '+month+'',fontsize=22)
plt.gca().xaxis.set_major_locator(ticker.MultipleLocator(40))
plt.xlim(0, 205)
#plt.xlim(0, 500)

#plt.ylim(0,10**7)
plt.tick_params(axis='both', which='major', labelsize=15)
yticks = [10**(-8), 10**(-7), 10**(-6), 10**(-5), 10**(-4), 10**(-3), 10**(-2), 10**(-1), 1]
#yticks = [10**(-7), 10**(-6), 10**(-5), 10**(-4), 10**(-3), 10**(-2), 10**(-1), 1]
plt.gca().set_yticks(yticks)
plt.gca().set_ylim(bottom=yticks[0], top=1)


min_ylim = plt.gca().get_ylim()[0]  # Get the minimum y-limit of the current axes

plt.scatter(np.max(cesm2), min_ylim + 0.000000003, color=colors[4], marker='*', s=200, label='CESM2 max', zorder=5)
plt.scatter(np.max(eobs), min_ylim + 0.000000003, color=colors[5], marker='*', s=200, label='E-OBS max', zorder=5)
plt.scatter(np.max(era5), min_ylim + 0.000000003, color=colors[1], marker='*', s=200, label='ERA5 max', zorder=5)
plt.scatter(np.max(ifs), min_ylim + 0.000000003, color=colors[2], marker='*', s=200, label='IFS max', zorder=5)
plt.scatter(np.max(icon), min_ylim + 0.000000003, color=colors[0], marker='*', s=200, label='ICON max', zorder=5)
plt.scatter(np.max(imerg), min_ylim + 0.000000003, color=colors[6], marker='*', s=200, label='IMERG max', zorder=5)


#plt.axvline(x=np.max(cesm2), color=colors[4], linestyle='--', linewidth=2, label = 'CESM2 max')
#plt.axvline(x=np.max(eobs), color=colors[5], linestyle='--', linewidth=2, label='E-OBS max')
#plt.axvline(x=np.max(era5), color = colors[1], linestyle = '--', linewidth=2, label = 'ERA5 max')
#plt.axvline(x=np.max(ifs), color = colors[2], linestyle = '--', linewidth=2, label = 'IFS max')
#plt.axvline(x=np.max(icon), color = colors[0], linestyle = '--', linewidth =2, label = 'ICON max')
#plt.axvline(x=np.max(imerg), color = colors[6], linestyle = '--', linewidth =2, label = 'IMERG max')

plt.legend(loc='upper right',prop={'size': 13})
plt.savefig('/home/jwille/images/Med_'+month+'_histo_cycle3_interp_cesm2.pdf', format='pdf',bbox_inches='tight')
plt.show()